# E-Commerce Recommendations

<b> Author:</b> Dominick DiTucci <br>
<b>email:</b> djditucci@gmail.com<br>
<br>
The premise of this project is a hypothetical company, "The Company", in the e-commerce industry that would like to develop a recommendation system. "The Company" specializes in selling adhesives and sealants in addition to many related products in other categories. Our purpose is to identify reasonable complements to items a customer is interested in by identifying other items that are often purchased in the same transaction as the item of interest. The reasonable complements can be recommended to the customer through the context of "customers who viewed ... also viewed..." and "customers who purchased ... also purchased..." for the purpose of identifying alternative items or add-on items.

The recommendations will be generated by using the apriori algorithm to extract frequent itemsets for association rule mining. The association rules will take the antecedent (item being viewed/purchased) and recommend associated consequents (complementary items) with the specified support and lift.

####  <u> Key Terms </u>
<ul><b>Association Rules:</b> Probability of association between items based on how frequently they are purchased together. If (X) purchased then (Y) has a higher probability of being purchased. </ul>
<ul><b>Antecedent:</b> Item being viewed or purchased. </ul> 
<ul><b>Consequent:</b> Other item(s) that are frequently purchased in the same transaction.</ul>
<ul><b>Support:</b> The relative frequency the antecedent and consequent show up in the same transactions.</ul>
<ul><b>Lift:</b> The ratio of the observed support to what is expected if the two rules were independent and the item association was completely random.</ul>
<ul><b>Confidence:</b> The reliability measure for the association rules.</ul>
<br>
The data used will be two datasets that have been combined. There is a dataset with six months of transactions where items in the "adhesives and sealants" category were purchased and another dataset containing all transactions over a two week period. The raw data contains:

<ul><b>Order Number:</b> The unique identifier for each individual transaction</ul>
<ul><b>l1:</b> Level 1 product identifier (most broad)</ul>
<ul><b>l2:</b> Level 2 product identifier</ul>
<ul><b>l3:</b> Level 3 product identifier (most granular)</ul>
<ul><b>Sku:</b> The unique identifier for a specific item</ul>
<ul><b>Brand:</b> The unique identifier for the specific brand of an item </ul>

In [1]:
# Import Libraries
import sqlite3
import pandas as pd
import numpy as np
#!pip install mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

### Data Preparation
Load the raw data into pandas dataframes and seperate into columns.

In [2]:
# Load txt files into pandas and separate columns 
adhesiveSealantTrans = pd.read_csv("/Users/DJ/E-Commerce-Recommendations/e-comm-data/Transactions with A&S.txt",sep='\t', header=0)
allTransaction = pd.read_csv("/Users/DJ/E-Commerce-Recommendations/e-comm-data/All Transations - 2 Weeks.txt", sep='\t', header=0)

In [3]:
# Verify the data loaded correctly
adhesiveSealantTrans.head(5)

,order_number,l1,l2,l3,sku,brand
0,1182478,"Adhesives, Sealants and Tape",Putties,Putties,122821,2180
1,63908,"Adhesives, Sealants and Tape",Caulks,Caulks and Sealants,276172,1054
2,1165840,"Adhesives, Sealants and Tape",Concrete and Asphalt,Concrete Cleaners and Sealers,110119,4326
3,928748,Material Handling,Casters and Wheels,Plate Casters,71052,1793
4,1088891,"Adhesives, Sealants and Tape",Glues and Cements,Retaining Compounds,251285,2515


In [4]:
# Verify the data loaded correctly
allTransaction.head(5)

,order_number,l1,l2,l3,sku,brand
0,168266,Power Tools,Power Saws and Accessories,Reciprocating Saw Blades,265105,2768
1,123986,Safety,Spill Control Supplies,Temporary Leak Repair,215839,586
2,158978,Hardware,Door Hardware,Thresholds,284756,1793
3,449035,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,12579,1231
4,781232,Motors,General Purpose AC Motors,General Purpose AC Motors,194681,2603


<br>
Both dataframes will be combined since the goal is recommending the items most frequently associated with each other regardless of category.

In [5]:
# Combine transaction data since adhesives/sealants do not need to be seperate from all other transactions
frames = [adhesiveSealantTrans, allTransaction]
allTransactions = pd.concat(frames)

# Check data loaded correctly
allTransactions.head()

,order_number,l1,l2,l3,sku,brand
0,1182478,"Adhesives, Sealants and Tape",Putties,Putties,122821,2180
1,63908,"Adhesives, Sealants and Tape",Caulks,Caulks and Sealants,276172,1054
2,1165840,"Adhesives, Sealants and Tape",Concrete and Asphalt,Concrete Cleaners and Sealers,110119,4326
3,928748,Material Handling,Casters and Wheels,Plate Casters,71052,1793
4,1088891,"Adhesives, Sealants and Tape",Glues and Cements,Retaining Compounds,251285,2515


In [6]:
# See final shape of combined dataset
allTransactions.shape

(5494816, 6)

In [7]:
# Check data types
print("All Transactions Data", allTransactions.dtypes)

All Transactions Data order_number     int64
l1              object
l2              object
l3              object
sku              int64
brand            int64
dtype: object


## Exploratory  Analysis

### SQL Queries

Brief analysis will be done using SQL queries. First, the database will be created. Then the database will be connected and loaded with the combined data.

In [8]:
# Create database
db = sqlite3.connect("transactions.db")

# Connect to database
con = sqlite3.connect("/Users/DJ/E-Commerce-Recommendations/transactions.db")

In [9]:
# Move dataframe to sql table in transactions.db
allTransactions.to_sql("all_trans", con, if_exists="replace", index=False)

In [10]:
# See unique values for all features
print("Unique Order Numbers: \n", pd.read_sql_query("SELECT COUNT(DISTINCT order_number) FROM all_Trans;", con))
print("\nUnique Brands: \n", pd.read_sql_query("SELECT COUNT(DISTINCT Brand) FROM all_Trans;", con))
print("\nUnique Sku: \n", pd.read_sql_query("SELECT COUNT(DISTINCT Sku) FROM all_Trans;", con))
print("\nUnique l1: \n", pd.read_sql_query("SELECT COUNT(DISTINCT l1) FROM all_Trans;", con))
print("\nUnique l2: \n", pd.read_sql_query("SELECT COUNT(DISTINCT l2) FROM all_Trans;", con))
print("\nUnique l3: \n", pd.read_sql_query("SELECT COUNT(DISTINCT l3) FROM all_Trans;", con))

Unique Order Numbers: 
    COUNT(DISTINCT order_number)
0                       1243048

Unique Brands: 
    COUNT(DISTINCT Brand)
0                   4841

Unique Sku: 
    COUNT(DISTINCT Sku)
0               333127

Unique l1: 
    COUNT(DISTINCT l1)
0                  33

Unique l2: 
    COUNT(DISTINCT l2)
0                 593

Unique l3: 
    COUNT(DISTINCT l3)
0                6355


Let's verify there are no missing values.

In [11]:
# Check all_trans for missing data
print("\nAll Transactions Missing Data")
print("Brand", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN brand is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM all_trans;", con))
print("Sku", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN sku is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM all_trans;", con))
print("L1", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN l1 is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM all_trans;", con))
print("L2", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN l2 is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM all_trans;", con))
print("L3", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN l3 is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM all_trans;", con))



All Transactions Missing Data
Brand    ProportionMissing
0                0.0
Sku    ProportionMissing
0                0.0
L1    ProportionMissing
0                0.0
L2    ProportionMissing
0                0.0
L3    ProportionMissing
0                0.0


There is no missing data.

What are the 10 most-purchased items?

In [12]:
# See top 10 items in ALL transactons
pd.read_sql_query("SELECT l1,l2, l3, COUNT(sku) FROM all_trans GROUP BY sku ORDER BY COUNT(sku) DESC LIMIT 10;", con)

,l1,l2,l3,COUNT(sku)
0,"Adhesives, Sealants and Tape",Tapes,Electrical Tapes,35812
1,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,30079
2,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,25550
3,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,10981
4,"Adhesives, Sealants and Tape",Thread and Gasket Sealants,Pipe Sealant Tape,10920
5,"Adhesives, Sealants and Tape",Tapes,Duct and Cloth Tapes,10162
6,"Adhesives, Sealants and Tape",Tapes,Duct and Cloth Tapes,8549
7,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,8468
8,Cleaning,Paper Products and Dispensers,"Paper Towels, Rolls",8288
9,"Adhesives, Sealants and Tape",Thread and Gasket Sealants,Pipe Sealant Tape,8229


Electrical tape is the most-purchased item with almost 36,000 units in the transaction data. Batteries are the next most-purchased items, followed by pipe sealant tape.



## Building a recommendation system

Given the limited amount of data available (unique identifiers and categories only), we will find frequent itemsets using the apriori algorithm to find which items are purchased together more often than expected. All items will be grouped by their order number to identify what was purchased in the same transaction. We will then use the most-granular (l3) product hierarchy to find what each item is and make recommendations based on that. The broader product hierarchy levels will be dropped along with the numerical unique identifiers (brand and sku) as they are not useful for our purposes. 

### Prepare Data

We know the shape of the dataframe is (5494816, 6) before altering it.

Sku and brand will be dropped since we do not have information on which items and brands these numbers correspond to. The l1 and l2 categories will also be dropped because we need the most granular (l3) data available to make meaningful recommendations.

In [13]:
# Drop sku and brand 
allTransactions.drop('sku', axis=1, inplace=True)
allTransactions.drop('brand', axis=1, inplace=True)

# Drop most broad categories and keep most granular
allTransactions.drop('l1', axis=1, inplace=True)
allTransactions.drop('l2', axis=1, inplace=True)

Now the data will be grouped by order number so all items purchased in the same transaction are in a list together. The length of the list should match the count of unique order numbers (1243048).

In [14]:
# Combine items into lists by order number
grouped = allTransactions.groupby('order_number')['l3'].apply(list)
grouped.head()

order_number
1                     [Steel-Toe Work Boots and Shoes]
2    [Bib Overalls, Traffic Safety Vests, Work Cove...
3                                          [Rainsuits]
4                     [Steel-Toe Work Boots and Shoes]
5                     [Steel-Toe Work Boots and Shoes]
Name: l3, dtype: object

In [15]:
# Check shape of list 
grouped.shape

(1243048,)

There are over 1 million orders. We will filter out orders with less than 10 items to make our data more relevant for finding items that are purchased together.

In [16]:
#Filter out small orders
filtered_group = [x for x in grouped if len(x)>=10 ]

# Sample first few large orders
filtered_group[:3]

[['Bib Overalls',
  'Traffic Safety Vests',
  'Work Coveralls',
  'Bib Overalls',
  'Traffic Safety Vests',
  'Traffic Safety Vests',
  'Bib Overalls',
  'Headbands, Sweatbands and Cooling Headwear',
  'Hard Hats',
  'Hard Hat Accessories',
  'Traffic Warning Flags and Banners',
  'Traffic Warning Flags and Banners',
  'Bib Overalls',
  'Hard Hats',
  'High Visibility Shirts and Sweatshirts'],
 ['Cut-Resistant Gloves',
  'Cut-Resistant Gloves',
  'Leather Work Gloves',
  'Sports Drinks and Thirst Quenchers',
  'Safety Glasses',
  'Leather Work Gloves',
  'Leather Work Gloves',
  'Leather Work Gloves',
  'Leather Work Gloves',
  'Leather Work Gloves',
  'Leather Work Gloves',
  'Leather Work Gloves',
  'Leather Work Gloves',
  'Leather Work Gloves',
  'Leather Work Gloves',
  'Standard Batteries',
  'Standard Batteries',
  'Standard Batteries',
  'Standard Batteries',
  'Standard Batteries',
  'Standard Batteries',
  'Safety Glasses',
  'Safety Glasses',
  'Safety Glasses',
  'Cut-Resis

Now the data can be one-hot encoded using the MLxtend library.

In [17]:
# One-hot encode data in pandas dataframe
te = TransactionEncoder()
te_ary = te.fit(filtered_group).transform(filtered_group)
transaction_group = pd.DataFrame(te_ary, columns=te.columns_)
transaction_group.head()

,12 Volt Accessories,12-Point Flange Head Cap Screws,3-Ring Binder Accessories,3-Ring Binders,3.3 Inch Diameter Motors,3D Printing Materials,4-Way and Box Parallels,4.4 Inch Diameter Motors,5 X 20mm Glass and Ceramic Fuses,50 Hz Motors,...,Wrist Watches,Writing Instrument Accessories,Y Strainers,Yard Hydrants,Yoke Ends,Zone Valve Accessories,Zone Valve Actuators,Zone Valves,Zoning Dampers,pH Meters
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Find itemsets that are in at least 1% of our data:

In [18]:
# Find items with minimal support using apriori algorithm to show which have any possible relationship
apriori(transaction_group, min_support=0.01, use_colnames=True)

,support,itemsets
0,0.055478,(Abrasive Cut-Off and Chop Wheels)
1,0.020519,(Abrasive Rolls)
2,0.030859,(Absorbent Pads)
3,0.014272,(Adhesive Bandages)
4,0.019328,(Adhesives)
5,0.024291,(Adjustable Wrenches)
6,0.010042,(Aerosol Dusters)
7,0.021390,(Air Guns)
8,0.013432,(American Glass and Ceramic Fuses)
9,0.043100,(Anti-Seize Compounds)


To remove any orders with only 1 item purchased, we'll create a column with the number of items in each order and then filter out single-item transactions.

In [19]:
# Find frequent item sets and and number of items in each order
freq_itemsets = apriori(transaction_group, min_support=0.01, use_colnames=True)
freq_itemsets['length'] = freq_itemsets['itemsets'].apply(lambda x : len(x))
freq_itemsets.head()

,support,itemsets,length
0,0.055478,(Abrasive Cut-Off and Chop Wheels),1
1,0.020519,(Abrasive Rolls),1
2,0.030859,(Absorbent Pads),1
3,0.014272,(Adhesive Bandages),1
4,0.019328,(Adhesives),1


In [20]:
#Filter out single-item orders 
freq_itemsets[ (freq_itemsets['length'] > 1) &
             (freq_itemsets['support'] > 0.02) ]

,support,itemsets,length
302,0.034891,"(Black Pipe Fittings, Black Pipe)",2
315,0.021764,"(Flat Washers, Black Pipe Fittings)",2
330,0.029003,"(Disposable Gloves, Cable Ties)",2
332,0.028087,"(Duct and Cloth Tapes, Cable Ties)",2
334,0.053065,"(Electrical Tapes, Cable Ties)",2
336,0.036494,"(Flat Washers, Cable Ties)",2
338,0.030003,"(Hex Head Cap Screws, Cable Ties)",2
339,0.024398,"(Hex Nuts, Cable Ties)",2
341,0.031103,"(Jobber Length Drill Bits, Cable Ties)",2
348,0.027453,"(Pipe Sealant Tape, Cable Ties)",2


## Create Recommendations
Now we can generate a dataframe of association rules and their corresponding metrics. We will focus on the support, lift, and confidence of each association. This will ensure that the associations our recommendations will be based on occur at a relative frequency (support) that is higher than expected with random associations (lift) to some degree of confidence.

In [28]:
# Generate association rules to make recommendations using lift metric to choose items that are purchased together 
# more often than expected
rules = association_rules(freq_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Abrasive Cut-Off and Chop Wheels),(Cable Ties),0.055478,0.137088,0.012081,0.217756,1.588438,0.004475,1.103124
1,(Cable Ties),(Abrasive Cut-Off and Chop Wheels),0.137088,0.055478,0.012081,0.088124,1.588438,0.004475,1.035800
2,(Abrasive Cut-Off and Chop Wheels),(Depressed Center Wheels),0.055478,0.042527,0.013287,0.239504,5.631817,0.010928,1.259012
3,(Depressed Center Wheels),(Abrasive Cut-Off and Chop Wheels),0.042527,0.055478,0.013287,0.312444,5.631817,0.010928,1.373738
4,(Abrasive Cut-Off and Chop Wheels),(Duct and Cloth Tapes),0.055478,0.171658,0.012256,0.220922,1.286988,0.002733,1.063234


Generally, a lift above 1 can infer some association. We will filter out associations that have a lift of at least 6 and confidence of at least 80% to find the most relevant recommendations.

In [29]:
# Filter items and save
saved_recs = rules[ (rules['lift'] >= 6) &
     rules['confidence'] >= 0.8]

# Check saved dataframe
saved_recs.sort_values(by=['confidence'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2949,"(Lock Nuts, Hex Head Cap Screws, Hex Nuts, Loc...",(Flat Washers),0.012218,0.116974,0.011447,0.936875,8.009248,0.010018,13.988528
2922,"(Lock Nuts, Hex Nuts, Lock Washers)",(Flat Washers),0.014219,0.116974,0.013127,0.923201,7.892349,0.011464,11.497861
2893,"(Lock Nuts, Hex Head Cap Screws, Lock Washers)",(Flat Washers),0.017495,0.116974,0.015899,0.908773,7.769010,0.013852,9.679484
2840,"(Hex Head Cap Screws, Hex Nuts, Lock Washers)",(Flat Washers),0.029011,0.116974,0.026109,0.899974,7.693782,0.022715,8.827935
2825,"(Lock Nuts, Hex Head Cap Screws, Hex Nuts)",(Flat Washers),0.018908,0.116974,0.016999,0.899031,7.685721,0.014787,8.745488
2287,"(Lock Nuts, Lock Washers)",(Flat Washers),0.021336,0.116974,0.018961,0.888690,7.597319,0.016465,7.933036
2908,"(Jobber Length Drill Bits, Hex Nuts, Lock Wash...",(Flat Washers),0.012676,0.116974,0.011248,0.887349,7.585858,0.009766,7.838625
2547,"(Hex Nuts, Lock Washers, Cable Ties)",(Flat Washers),0.012065,0.116974,0.010676,0.884810,7.564150,0.009264,7.665829
2531,"(Hex Head Cap Screws, Lock Washers, Cable Ties)",(Flat Washers),0.013913,0.116974,0.012180,0.875412,7.483804,0.010553,7.087547
2950,"(Lock Nuts, Flat Washers, Hex Nuts, Lock Washers)",(Hex Head Cap Screws),0.013127,0.100502,0.011447,0.872019,8.676589,0.010128,7.028347


The final step now that recommendations have been generated is to save them as a csv file for later use.

In [23]:
# Save Recs to csv
saved_recs.to_csv('recommendations.csv', index=False)

##  Summary

The apriori algorithm and association rule mining were used to extract frequent itemsets (items purchased together) from the data and generate rules for frequent item patterns. These rules can be used in a recommendation engine in the context of "customers who viewed ... also viewed..." and "customers who purchased ... also purchased..." in order to encourage add-on purchases or encourage past customers to  continue using "The Company" for their e-commerce needs. The results are contained in a CSV file that can be used across platforms for production purposes.